In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import gzip as gz
import pysam
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy
import glob
from joblib import Parallel, delayed
import sys
import argparse
import plotly.express as px
import plotly.io as pio
import os
from Bio.Seq import Seq
from Bio import SeqIO
import swifter
import sys
import plotly.express as px
from Levenshtein import distance
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import warnings
warnings.filterwarnings("ignore")

## two filters, old is constitutive exon, new is CDS 5% to 65%, sometimes overlap return very few gRNAs, so we need to find new gRNAs

In [2]:
df = pd.read_csv("design2_input.tsv",sep="\t")

In [3]:
df

,Gene,pool1_hit1,pool1_hit2,crispick,Rank
0,ADORA2A,NaN,NaN,GCGGCGGCCGACATCGCAGT,1
1,PINLYP,NaN,NaN,CATGGTGGTGGATATAACGC,2
2,TET2,NaN,NaN,GGAACCACAAAGCTAGCGTC,3
3,UBE2L3,GAAGCGGGTGCTCAGGCT,NaN,TATCATATGGAGGGTTGTCC,4
4,ELOB,CAGTCAAACAGCACGGCCAC,NaN,AGGACGTGTTCCTCATGATC,5
5,VHL,ATGGCACAAATAATTCAGTT,NaN,GAGATGCAGGGACACACGAT,6
6,KCTD5,GGACACGCTGCCAGGGCGC,NaN,AAGTGGGTCCGACTCAACGT,7
7,CUL5,AGCTTGTTTACATAATCCGC,NaN,AACATTGGCTCTATACCATT,8
8,BCL2L11,TGAGATAGTGGTTGAAGGCC,NaN,AGGTAGACAATTGCAGCCTG,9
9,UBAP2L,GGATTGGATGGCACCAAGAG,NaN,AAGTAGCCTGTATACCAGCA,10


In [19]:
hex = pd.read_csv("HEXEvent_hg38_all.tsv",sep="\t")

In [20]:
hex

,chromo,strand,start,end,count,alt3,alt5,alt3+5,skip,constitLevel,inclLevel,3usageLevel,5usageLevel,alt3singleCount,alt3singleLoc,alt5singleCount,alt5singleLoc,alt3and5singleCount,alt3and5singleLoc,OnlyESTexonCount,OnlyESTexons,genename
0,chr1,+,12612,12721,8,2,0,1,0,0.727,1.000,0.727,0.909,2,12594,0,#,1,12645-12697,0,#,onlyEST
1,chr1,-,14969,15038,16,0,0,0,1,0.941,0.941,1.000,1.000,0,#,0,#,0,#,2,"chr1-15004-15038,chr1-14969-15250",onlyEST
2,chr1,-,15795,15947,12,0,0,0,2,0.857,0.857,1.000,1.000,0,#,0,#,0,#,0,#,onlyEST
3,chr1,-,16606,16765,9,0,0,0,2,0.818,0.818,1.000,1.000,0,#,0,#,0,#,9,"chr1-16606-16768,chr1-16723-16765",onlyEST
4,chr1,-,16857,17055,67,0,1,0,2,0.957,0.971,1.000,0.985,0,#,1,16853,0,#,3,"chr1-16857-17058,chr1-16856-17058,chr1-16856-1...",onlyEST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227174,chrY,+,57210639,57210792,19,0,0,0,0,1.000,1.000,1.000,1.000,0,#,0,#,0,#,1,chrY+57210639-57210762,onlyEST
227175,chrY,+,57211551,57211620,33,0,1,0,0,0.971,1.000,1.000,0.971,0,#,1,57211569,0,#,0,#,onlyEST
227176,chrY,-,57212901,57213175,1,0,0,0,0,1.000,1.000,1.000,1.000,0,#,0,#,0,#,0,#,onlyEST
227177,chrY,-,57213855,57213964,2,0,0,0,0,1.000,1.000,1.000,1.000,0,#,0,#,0,#,0,#,onlyEST


In [6]:
set(df.Gene)-set(exon.genename)

{'COP1', 'CYRIB', 'ELOB', 'OTULINL', 'SOCS1'}

In [7]:
exon = pd.read_csv("/home/yli11/Data/Human/hg38/annotations/gencode_v47/gencode.v47.basic.exon.bed",sep="\t",header=None)

In [8]:
exon.head()

,0,1,2,3,4,5
0,chr1,11425,11671,ENSG00000290825.2,.,+
1,chr1,12009,12057,ENSG00000223972.6,.,+
2,chr1,12009,12227,ENSG00000290825.2,.,+
3,chr1,12178,12227,ENSG00000223972.6,.,+
4,chr1,12612,12697,ENSG00000223972.6,.,+


In [9]:
GID = pd.read_csv("/home/yli11/Data/Human/hg38/annotations/gencode_v47/gencode.v47.GID.GName.tsv",sep="\t")
GID.head()

,Gene_ID,Gene_Name
0,ENSG00000290825.2,DDX11L16
1,ENSG00000223972.6,DDX11L1
2,ENSG00000310526.1,WASH7P
3,ENSG00000227232.6,WASH7P
4,ENSG00000278267.1,MIR6859-1


In [14]:
exon[4] = exon[3].map(GID.set_index("Gene_ID").Gene_Name)

In [12]:
set(df.Gene)-set(exon[6])

set()

In [16]:
exon[[0,1,2,3,4,5]].to_csv("/home/yli11/Data/Human/hg38/annotations/gencode_v47/gencode.v47.basic.exon.bed",sep="\t",header=False,index=False)

In [17]:
exon.head()

,0,1,2,3,4,5,6
0,chr1,11425,11671,ENSG00000290825.2,DDX11L16,+,DDX11L16
1,chr1,12009,12057,ENSG00000223972.6,DDX11L1,+,DDX11L1
2,chr1,12009,12227,ENSG00000290825.2,DDX11L16,+,DDX11L16
3,chr1,12178,12227,ENSG00000223972.6,DDX11L1,+,DDX11L1
4,chr1,12612,12697,ENSG00000223972.6,DDX11L1,+,DDX11L1


In [23]:
exon[exon[1].between(12612-2,12612+2)]

,0,1,2,3,4,5,6,chromo,strand,start,end,count,alt3,alt5,alt3+5,skip,constitLevel,inclLevel,3usageLevel,5usageLevel,alt3singleCount,alt3singleLoc,alt5singleCount,alt5singleLoc,alt3and5singleCount,alt3and5singleLoc,OnlyESTexonCount,OnlyESTexons,genename
4,chr1,12612,12697,ENSG00000223972.6,DDX11L1,+,DDX11L1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,chr1,12612,12721,ENSG00000290825.2,DDX11L16,+,DDX11L16,chr1,+,12612.0,12721.0,8.0,2.0,0.0,1.0,0.0,0.727,1.0,0.727,0.909,2,12594,0,#,1,12645-12697,0.0,#,onlyEST
428499,chr18,12614,12763,ENSG00000262352.3,LINC02564,+,LINC02564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
chromo	strand	start	end

In [21]:
exon = exon.merge(hex,how="left",left_on=[0,5,1,2],right_on=['chromo','strand','start','end'])

In [24]:
exon = exon.drop([6,'chromo','strand','start','end','genename'],axis=1)

In [26]:
exon.to_csv("gencode.v47.basic.exon.HexEvent.tsv",sep="\t",index=False)

In [29]:
exon.head()

,0,1,2,3,4,5,count,alt3,alt5,alt3+5,skip,constitLevel,inclLevel,3usageLevel,5usageLevel,alt3singleCount,alt3singleLoc,alt5singleCount,alt5singleLoc,alt3and5singleCount,alt3and5singleLoc,OnlyESTexonCount,OnlyESTexons
0,chr1,11425,11671,ENSG00000290825.2,DDX11L16,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr1,12009,12057,ENSG00000223972.6,DDX11L1,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chr1,12009,12227,ENSG00000290825.2,DDX11L16,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,chr1,12178,12227,ENSG00000223972.6,DDX11L1,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,chr1,12612,12697,ENSG00000223972.6,DDX11L1,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df2 = pd.read_csv("gencode.v47.basic.exon.HexEvent.GeCKOv1.tsv",sep="\t",header=None)
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,chr1,11425,11671,ENSG00000290825.2,DDX11L16,+,.,-1,-1,.,-1,.,0
1,chr1,12009,12057,ENSG00000223972.6,DDX11L1,+,.,-1,-1,.,-1,.,0
2,chr1,12009,12227,ENSG00000290825.2,DDX11L16,+,.,-1,-1,.,-1,.,0
3,chr1,12178,12227,ENSG00000223972.6,DDX11L1,+,.,-1,-1,.,-1,.,0
4,chr1,12612,12697,ENSG00000223972.6,DDX11L1,+,.,-1,-1,.,-1,.,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009686,chrY,57214349,57214397,ENSG00000292371.1,DDX11L16,-,.,-1,-1,.,-1,.,0
1009687,chrY,57214349,57214567,ENSG00000310542.1,ENSG00000310542,-,.,-1,-1,.,-1,.,0
1009688,chrY,57214349,57214567,ENSG00000310542.1,ENSG00000310542,-,.,-1,-1,.,-1,.,0
1009689,chrY,57215366,57215561,ENSG00000310542.1,ENSG00000310542,-,.,-1,-1,.,-1,.,0


In [30]:
exon = exon.merge(df2,how="left",left_on=[0,5,1,2],right_on=[0,5,1,2])

In [33]:
exon['is_constitutive_exon.GeCKOv1'] = exon[12]>0

In [36]:
exon.columns.tolist()

[0,
 1,
 2,
 '3_x',
 '4_x',
 5,
 'count',
 'alt3',
 'alt5',
 'alt3+5',
 'skip',
 'constitLevel',
 'inclLevel',
 '3usageLevel',
 '5usageLevel',
 'alt3singleCount',
 'alt3singleLoc',
 'alt5singleCount',
 'alt5singleLoc',
 'alt3and5singleCount',
 'alt3and5singleLoc',
 'OnlyESTexonCount',
 'OnlyESTexons',
 '3_y',
 '4_y',
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 'is_constitutive_exon.GeCKOv1']

In [37]:
exon_final=exon[[0,
 1,
 2,
 '3_x',
 '4_x',
 5,
 'constitLevel',
 'is_constitutive_exon.GeCKOv1']]

In [39]:
exon_final.columns=["#chr",
 "start",
 "end",
 'GID',
 'GName',
 "strand",
 'HexEvent.constitLevel',
 'is_constitutive_exon.GeCKOv1']

In [ ]:
exon_final.to_csv("hg38.gencode_v47.basic.exon.info.tsv",sep="\t",index=False)